In [13]:
import time
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
import gensim.downloader as api

# Lade das vortrainierte Word2Vec-Modell
print("Downloading and loading the Word2Vec model...")
word2vec_model = api.load('word2vec-google-news-300')
print("Model loaded successfully.")

# Funktion zur Berechnung des durchschnittlichen Word2Vec-Vektors für einen Satz
def sentence_vector(sentence, model, vector_size):
    words = sentence.split()
    word_vectors = [model[word] for word in words if word in model]
    if len(word_vectors) == 0:
        return np.zeros(vector_size)  # Gib einen Nullvektor zurück, wenn keine Wörter im Modell gefunden werden
    return np.mean(word_vectors, axis=0)

# Definiere die Word2Vec-Ähnlichkeitsfunktion
def word2vec_similarity(text1, text2, model, vector_size):
    vec1 = sentence_vector(text1, model, vector_size)
    vec2 = sentence_vector(text2, model, vector_size)
    
    # Prüfe, ob einer der Vektoren nur aus Nullen besteht (d.h. keine gültigen Wörter gefunden)
    if np.all(vec1 == 0) or np.all(vec2 == 0):
        return 0  # Weise 0 Ähnlichkeit zu, wenn einer der Sätze leer ist oder keine gültigen Wörter enthält
    
    # Berechne die Kosinus-Ähnlichkeit
    return cosine_similarity([vec1], [vec2])[0, 0]

# Beispiel-Sätze für den Vergleich
satz1 = "This is a test sentence"
satz2 = "This is another example sentence"
satz3 = "Vectors represent words in Word2Vec"

# Lade den Beispieldatensatz
data = {
    'Satz 1': [satz1],
    'Satz 2': [satz2],
    'Satz 3': [satz3]
}

df = pd.DataFrame(data)

# Bereite die Ergebnisse vor
results = []

# Iteriere über jede Zeile, um die Sätze zu vergleichen
for index, row in df.iterrows():
    # Extrahiere die Sätze aus jeder Spalte
    satz1 = row.get('Satz 1', "")
    satz2 = row.get('Satz 2', "")
    satz3 = row.get('Satz 3', "")

    # Liste der Vergleiche: Satz 1 mit Satz 2, Satz 1 mit Satz 3, Satz 2 mit Satz 3
    comparisons = [("Satz 1 mit Satz 2", satz1, satz2), 
                   ("Satz 1 mit Satz 3", satz1, satz3),
                   ("Satz 2 mit Satz 3", satz2, satz3)]
    
    # Vergleiche jedes Satzpaar und berechne die Word2Vec-Kosinus-Ähnlichkeit
    for comparison_label, text1, text2 in comparisons:
        if not text1 or not text2:
            continue
        
        start_time = time.perf_counter()
        similarity = word2vec_similarity(text1, text2, word2vec_model, vector_size=300)  # Google News Word2Vec verwendet 300 Dimensionen
        end_time = time.perf_counter()
        
        elapsed_time_ms = round((end_time - start_time) * 1000, 9)
        
        results.append({
            "Vergleich": comparison_label,
            "Word2Vec Ähnlichkeit (Cosine)": similarity,
            "Berechnungszeit (ms)": elapsed_time_ms
        })

# Erstelle ein DataFrame aus den Ergebnissen
results_df = pd.DataFrame(results)

# Drucke die Ergebnisse in einem formatierten DataFrame
print(results_df.to_string(index=False))

Vergleich zwischen Text 1 und Text 2:
Text 1: Der Himmel ist blau und die Sonne scheint hell. Der Wind weht sanft durch die Bäume.
Text 2: Der Himmel ist nicht blau und die Sonne scheint nicht hell. Der Wind weht nicht sanft durch die Bäume.
Kosinus-Ähnlichkeit: 0.9658278292827439

Vergleich zwischen Text 1 und Text 3:
Text 1: Der Himmel ist blau und die Sonne scheint hell. Der Wind weht sanft durch die Bäume.
Text 3: Der Himmel zeigt sich in Blau, und die Sonne strahlt stark. Ein sanfter Wind bewegt die Bäume
Kosinus-Ähnlichkeit: 0.9342922266833731

Vergleich zwischen Text 2 und Text 3:
Text 2: Der Himmel ist nicht blau und die Sonne scheint nicht hell. Der Wind weht nicht sanft durch die Bäume.
Text 3: Der Himmel zeigt sich in Blau, und die Sonne strahlt stark. Ein sanfter Wind bewegt die Bäume
Kosinus-Ähnlichkeit: 0.9004386113439647


In [31]:
import os
import time
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# Load the pre-trained PromptBERT model (you can choose other sentence-transformer models if needed)
print("Loading the PromptBERT model...")
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
print("Model loaded successfully.")

# Function to calculate PromptBERT embeddings for a sentence
def sentence_embedding(sentence, model):
    return model.encode(sentence, convert_to_tensor=True)

# Define the PromptBERT similarity function
def promptbert_similarity(text1, text2, model):
    emb1 = sentence_embedding(text1, model)
    emb2 = sentence_embedding(text2, model)
    
    # Calculate cosine similarity
    similarity = cosine_similarity([emb1], [emb2])[0, 0]
    
    return similarity

# Load the CSV file with test sentences
df = pd.read_csv('testsätze.csv')

# Prepare results
results = []

# Iterate over each row to compare the sentences
for index, row in df.iterrows():
    # Extract the sentences from each column
    satz1 = row.get('Satz 1', "")
    satz2 = row.get('Satz 2', "")
    satz3 = row.get('Satz 3', "")

    # List of comparisons: Satz 1 with Satz 2, Satz 1 with Satz 3, Satz 2 with Satz 3
    comparisons = [("Satz 1 mit Satz 2", satz1, satz2), 
                   ("Satz 1 mit Satz 3", satz1, satz3),
                   ("Satz 2 mit Satz 3", satz2, satz3)]
    
    # Compare each sentence pair and calculate the PromptBERT cosine similarity
    for comparison_label, text1, text2 in comparisons:
        if not text1 or not text2:
            continue
        
        start_time = time.perf_counter()
        similarity = promptbert_similarity(text1, text2, model)
        end_time = time.perf_counter()
        
        elapsed_time_ms = round((end_time - start_time) * 1000, 9)
        
        results.append({
            "Vergleich": comparison_label,
            "PromptBERT Ähnlichkeit (Cosine)": similarity,
            "Berechnungszeit (ms)": elapsed_time_ms
        })

# Create a DataFrame from the results
results_df = pd.DataFrame(results)

# Print the results in a formatted DataFrame
print(results_df.to_string(index=False))

Vergleich zwischen Text 1 und Text 2:
Text 1: Der Himmel ist blau und die Sonne scheint hell. Der Wind weht sanft durch die Bäume.
Text 2: Der Himmel ist nicht blau und die Sonne scheint nicht hell. Der Wind weht nicht sanft durch die Bäume.
Kosinus-Ähnlichkeit: 0.9787534623563109

Prompts.A = 'Summarize sentence "{text}" in one word:"'
Prompts.B = 'You can only output one word. Summarize "{text}":"'
Prompts.C = 'Represent this sentence for searching relevant passages: {text}'
Vergleich zwischen Text 1 und Text 3:
Text 1: Der Himmel ist blau und die Sonne scheint hell. Der Wind weht sanft durch die Bäume.
Text 3: Der Himmel zeigt sich in Blau, und die Sonne strahlt stark. Ein sanfter Wind bewegt die Bäume.
Kosinus-Ähnlichkeit: 0.9578784755546029

Prompts.A = 'Summarize sentence "{text}" in one word:"'
Prompts.B = 'You can only output one word. Summarize "{text}":"'
Prompts.C = 'Represent this sentence for searching relevant passages: {text}'
Vergleich zwischen Text 2 und Text 3:
Text 2

In [ ]:
import torch
from sentence_transformers import SentenceTransformer
import torch.nn.functional as F

# Festlegen der Dimension für die Embeddings
matryoshka_dim = 1024  # Optionen: 64, 128, 256, 512, 768, 1024

# Laden des Modells
model = SentenceTransformer("aari1995/German_Semantic_V3", trust_remote_code=True, truncate_dim=matryoshka_dim)

# Sätze im gegebenen Format
texts = [
    {'text': 'Der Himmel ist blau und die Sonne scheint hell. Der Wind weht sanft durch die Bäume.'},
    {'text': 'Der Himmel ist nicht blau und die Sonne scheint nicht hell. Der Wind weht nicht sanft durch die Bäume.'},
    {'text': 'Der Himmel zeigt sich in Blau, und die Sonne strahlt stark. Ein sanfter Wind bewegt die Bäume'},
]

# Extrahiere die Sätze aus dem Wörterbuch
sentences = [item['text'] for item in texts]

# Berechnen der Embeddings (Verwenden von FP16 für weniger Speicherverbrauch)
embeddings = model.encode(sentences, convert_to_tensor=True).half()

# Berechne die Kosinus-Ähnlichkeit zwischen den Satz-Embeddings
similarities = F.cosine_similarity(embeddings.unsqueeze(1), embeddings.unsqueeze(0), dim=-1)

# Ausgabe der Kosinus-Ähnlichkeit im gewünschten Format
for i in range(len(sentences)):
    for j in range(i+1, len(sentences)):
        print(f'Similarity between text {i+1} and text {j+1}: {similarities[i, j].item():.4f}')

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
from angle_emb import AnglE, Prompts
from angle_emb.utils import cosine_similarity

#funktioniert
#https://github.com/SeanLee97/AnglE
#Infer LLM-based Models

# 1. Lade das Hauptmodell und die LoRA-Gewichte auf der CPU
model = AutoModelForCausalLM.from_pretrained('NousResearch/Llama-2-7b-hf', torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained('NousResearch/Llama-2-7b-hf')

# Lade das LoRA-Modell auf der CPU
lora_model = PeftModel.from_pretrained(model, 'SeanLee97/angle-llama-7b-nli-v2', torch_dtype=torch.float16)

# 2. Initialisiere AnglE mit dem Modell und den LoRA-Gewichten (ohne CUDA)
angle = AnglE.from_pretrained(
    'NousResearch/Llama-2-7b-hf',
    pretrained_lora_path='SeanLee97/angle-llama-7b-nli-v2',
    pooling_strategy='last',
    is_llm=True,
    torch_dtype=torch.float16
)

texts = [
    {'text': 'Der Himmel ist blau und die Sonne scheint hell. Der Wind weht sanft durch die Bäume.'},
    {'text': 'Der Himmel ist nicht blau und die Sonne scheint nicht hell. Der Wind weht nicht sanft durch die Bäume.'},
    {'text': 'Der Himmel zeigt sich in Blau, und die Sonne strahlt stark. Ein sanfter Wind bewegt die Bäume'},
]

doc_vecs = angle.encode(texts, prompt=Prompts.A)

# 4. Berechne die Kosinus-Ähnlichkeiten zwischen den Texten
for i, dv1 in enumerate(doc_vecs):
    for j, dv2 in enumerate(doc_vecs[i+1:], start=i+1):
        similarity = cosine_similarity(dv1, dv2)
        print(f'Similarity between text {i+1} and text {j+1}: {similarity.item()}')

# 5. Liste alle vordefinierten Prompts auf
print('All predefined prompts:', Prompts.list_prompts())